In [1]:
# Set up autoreloading of modules so that I can debug code in external files
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

import bydatetime
import hillpylib
from pandas import Timestamp

# Let's check what version of pandas, numpy and matplotlib we are using
print ("pandas version ", pd.__version__)
print ("numpy version ", np.version.version)
print ("matplotlib version ", mp.__version__)

pandas version  0.15.2
numpy version  1.9.2
matplotlib version  1.4.3


# Put it all together

Below I've strung together all the pieces to do an entire Hillmaker run. Change inputs as needed (e.g. scenario_name and associated parameter values) and run all the cells below. You can skip rereading the main input file if that isn't changing.

## Read main stop data file

In [3]:
file_stopdata = 'data/ShortStay.csv'
df = pd.read_csv(file_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59877 entries, 0 to 59876
Data columns (total 4 columns):
PatID        59877 non-null int64
InRoomTS     59877 non-null datetime64[ns]
OutRoomTS    59877 non-null datetime64[ns]
PatType      59877 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 2.3+ MB


## Set input parameters

In [4]:
scenario_name = 'sstest_60'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis = '1/1/1996'
end_analysis = '3/30/1996 23:45'
bin_size_mins = 60

# This next field wasn't in original Hillmaker. Use it to specify the name to use for the overall totals.
# At this point the totals actually aren't being calculated.
tot_fld_name = 'Total'

## Convert string dates to actual datetimes
start_analysis_dt = pd.Timestamp(start_analysis)
end_analysis_dt = pd.Timestamp(end_analysis)

# Mapper from weekday integer to string
daynum_to_dayname = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}


## Create the by datetime table

In [5]:
bydt_df = bydatetime.make_bydatetime(df,
                                     in_fld_name,
                                     out_fld_name,
                                     cat_fld_name,
                                     start_analysis_dt,
                                     end_analysis_dt,
                                     tot_fld_name,
                                     bin_size_mins)

rng_bydt created: 0.001239221001014812
found unique categories: 0.00917574500090268
Seeded bydatetime DataFrame created: 0.04094858499956899
dayofweek, bin_of_day, bin_of_week computed: 0.3417036080008984
Multi-index on bydatetime DataFrame created: 0.3448973089998617
Multi-index fully lexsorted: 0.348313488999338
Latest edits 1:36p
Num inner: 19795
Done processing 19795 stop recs: 14.604598367999643


In [6]:
bydt_df.dtypes

category               object
datetime       datetime64[ns]
arrivals              float64
departures            float64
occupancy             float64
day_of_week             int64
bin_of_day              int64
bin_of_week             int64
dtype: object

## Compute summary stats

In [7]:
def get_occstats(group, stub=''):
    return {stub+'count': group.count(), stub+'mean': group.mean(), 
            stub+'min': group.min(),
            stub+'max': group.max(), 'stdev': group.std(), 
            stub+'p50': group.quantile(0.5), stub+'p55': group.quantile(0.55),
            stub+'p60': group.quantile(0.6), stub+'p65': group.quantile(0.65),
            stub+'p70': group.quantile(0.7), stub+'p75': group.quantile(0.75),
            stub+'p80': group.quantile(0.8), stub+'p85': group.quantile(0.85),
            stub+'p90': group.quantile(0.9), stub+'p95': group.quantile(0.95),
            stub+'p975': group.quantile(0.975), 
            stub+'p99': group.quantile(0.99)}

In [8]:
bydt_dfgrp2 = bydt_df.groupby(['category','day_of_week','bin_of_day'])

occ_stats = bydt_dfgrp2['occupancy'].apply(get_occstats)
arr_stats = bydt_dfgrp2['arrivals'].apply(get_occstats)
dep_stats = bydt_dfgrp2['departures'].apply(get_occstats)

occ_stats_summary = occ_stats.unstack()
arr_stats_summary = arr_stats.unstack()
dep_stats_summary = dep_stats.unstack()



## Write summaries and by datetime out to CSV

In [9]:
file_bydt_csv = 'testing/bydate_' + scenario_name + '.csv'
bydt_df.to_csv(file_bydt_csv, index=False)

file_occ_csv = 'testing/occ_stats_' + scenario_name + '.csv'
file_arr_csv = 'testing/arr_stats_' + scenario_name + '.csv'
file_dep_csv = 'testing/dep_stats_' + scenario_name + '.csv'

occ_stats_summary.to_csv(file_occ_csv)
arr_stats_summary.to_csv(file_arr_csv)
dep_stats_summary.to_csv(file_dep_csv)

## Debugging

In [10]:
ts = pd.Timestamp('19960103 00:00:00')
print(ts)

1996-01-03 00:00:00


In [11]:
df_ART = df[(df.PatType == 'ART') & (df.InRoomTS < ts)]

In [12]:
df_ART.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 9 to 174
Data columns (total 4 columns):
PatID        30 non-null int64
InRoomTS     30 non-null datetime64[ns]
OutRoomTS    30 non-null datetime64[ns]
PatType      30 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 1.2+ KB


In [13]:
df_ART

,PatID,InRoomTS,OutRoomTS,PatType
9,10,1996-01-02 06:35:00,1996-01-02 08:30:00,ART
12,13,1996-01-02 06:55:00,1996-01-02 08:45:00,ART
15,16,1996-01-02 07:18:00,1996-01-02 08:53:00,ART
17,18,1996-01-02 07:23:00,1996-01-02 08:30:00,ART
21,22,1996-01-02 07:33:00,1996-01-02 08:45:00,ART
24,25,1996-01-02 07:40:00,1996-01-02 08:55:00,ART
25,26,1996-01-02 07:50:00,1996-01-02 10:10:00,ART
27,28,1996-01-02 07:52:00,1996-01-02 09:10:00,ART
30,31,1996-01-02 08:00:00,1996-01-02 09:00:00,ART
31,32,1996-01-02 08:02:00,1996-01-02 10:00:00,ART


In [14]:
bydt_df.head()

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-01 00:00:00      ART 1996-01-01 00:00:00         0   
         1996-01-01 01:00:00      ART 1996-01-01 01:00:00         0   
         1996-01-01 02:00:00      ART 1996-01-01 02:00:00         0   
         1996-01-01 03:00:00      ART 1996-01-01 03:00:00         0   
         1996-01-01 04:00:00      ART 1996-01-01 04:00:00         0   

                              departures  occupancy  day_of_week  bin_of_day  \
category datetime                                                              
ART      1996-01-01 00:00:00           0          0            0           0   
         1996-01-01 01:00:00           0          0            0           1   
         1996-01-01 02:00:00           0          0            0           2   
         1996-01-01 03:00:00           0          0            0           3   
         1996-01-01 04:00:00           0          0            0           4   

                              bin_of_week  
category datetime                          
ART      1996-01-01 00:00:00            0  
         1996-01-01 01:00:00            1  
         1996-01-01 02:00:00            2  
         1996-01-01 03:00:00            3  
         1996-01-01 04:00:00            4

In [15]:
bydt_df[25:50]

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-02 01:00:00      ART 1996-01-02 01:00:00         0   
         1996-01-02 02:00:00      ART 1996-01-02 02:00:00         0   
         1996-01-02 03:00:00      ART 1996-01-02 03:00:00         0   
         1996-01-02 04:00:00      ART 1996-01-02 04:00:00         0   
         1996-01-02 05:00:00      ART 1996-01-02 05:00:00         0   
         1996-01-02 06:00:00      ART 1996-01-02 06:00:00         2   
         1996-01-02 07:00:00      ART 1996-01-02 07:00:00         6   
         1996-01-02 08:00:00      ART 1996-01-02 08:00:00         5   
         1996-01-02 09:00:00      ART 1996-01-02 09:00:00         5   
         1996-01-02 10:00:00      ART 1996-01-02 10:00:00         2   
         1996-01-02 11:00:00      ART 1996-01-02 11:00:00         5   
         1996-01-02 12:00:00      ART 1996-01-02 12:00:00         1   
         1996-01-02 13:00:00      ART 1996-01-02 13:00:00         3   
         1996-01-02 14:00:00      ART 1996-01-02 14:00:00         0   
         1996-01-02 15:00:00      ART 1996-01-02 15:00:00         1   
         1996-01-02 16:00:00      ART 1996-01-02 16:00:00         0   
         1996-01-02 17:00:00      ART 1996-01-02 17:00:00         0   
         1996-01-02 18:00:00      ART 1996-01-02 18:00:00         0   
         1996-01-02 19:00:00      ART 1996-01-02 19:00:00         0   
         1996-01-02 20:00:00      ART 1996-01-02 20:00:00         0   
         1996-01-02 21:00:00      ART 1996-01-02 21:00:00         0   
         1996-01-02 22:00:00      ART 1996-01-02 22:00:00         0   
         1996-01-02 23:00:00      ART 1996-01-02 23:00:00         0   
         1996-01-03 00:00:00      ART 1996-01-03 00:00:00         0   
         1996-01-03 01:00:00      ART 1996-01-03 01:00:00         0   

                              departures  occupancy  day_of_week  bin_of_day  \
category datetime                                                              
ART      1996-01-02 01:00:00           0   0.000000            1           1   
         1996-01-02 02:00:00           0   0.000000            1           2   
         1996-01-02 03:00:00           0   0.000000            1           3   
         1996-01-02 04:00:00           0   0.000000            1           4   
         1996-01-02 05:00:00           0   0.000000            1           5   
         1996-01-02 06:00:00           0   0.500000            1           6   
         1996-01-02 07:00:00           0   4.400000            1           7   
         1996-01-02 08:00:00           6   9.716667            1           8   
         1996-01-02 09:00:00           4   6.933333            1           9   
         1996-01-02 10:00:00           5   4.833333            1          10   
         1996-01-02 11:00:00           2   5.516667            1          11   
         1996-01-02 12:00:00           6   5.016667            1          12   
         1996-01-02 13:00:00           3   2.900000            1          13   
         1996-01-02 14:00:00           1   2.516667            1          14   
         1996-01-02 15:00:00           2   1.483333            1          15   
         1996-01-02 16:00:00           0   1.000000            1          16   
         1996-01-02 17:00:00           1   0.000000            1          17   
         1996-01-02 18:00:00           0   0.000000            1          18   
         1996-01-02 19:00:00           0   0.000000            1          19   
         1996-01-02 20:00:00           0   0.000000            1          20   
         1996-01-02 21:00:00           0   0.000000            1          21   
         1996-01-02 22:00:00           0   0.000000            1          22   
         1996-01-02 23:00:00           0   0.000000            1          23   
         1996-01-03 00:00:00           0   0.000000            2           0   
         1996-01-03 01:00:00   